In [ ]:
from datetime import datetime
import os

import aioinflux
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
import numpy as np
import pandas as pd

In [ ]:
token_file = os.path.join(os.path.expanduser("~/"), ".summit_efd")
with open(token_file, 'r') as fd:
    uname = fd.readline().strip()  # Can't hurt to be paranoid
    pwd = fd.readline().strip()

In [ ]:
client = aioinflux.InfluxDBClient(host='summit-influxdb-efd.lsst.codes',
                                  port=443,
                                  ssl=True,
                                  username=uname,
                                  password=pwd,
                                  db='efd',
                                  output="dataframe")

In [ ]:
csc = "Environment"
csc_index = 1
last = False
d1 = datetime(2019, 9, 13)
d2 = datetime(2019, 9, 16)
time_window = 1.0  # seconds

In [ ]:
query_base = "SELECT {0} FROM \"efd\".\"autogen\".\"lsst.sal.{1}\""
query_time_range = "WHERE time >= \'{2}\' AND time <= \'{3}\'"
query_time_last = "ORDER BY time DESC LIMIT 1"

In [ ]:
if last:
    query = query_base + " " + query_time_last
else:
    query = query_base + " " + query_time_range

In [ ]:
summary_state_cols = "private_sndStamp,summaryState"
if csc_index > 0:
    summary_state_cols += f",{csc}ID"
ss_df = await client.query(query.format(summary_state_cols, f"{csc}.logevent_summaryState", d1, d2))

In [ ]:
error_code_cols = "private_sndStamp,errorCode,errorReport,traceback"
if csc_index > 0:
    error_code_cols += f",{csc}ID"
ec_df = await client.query(query.format(error_code_cols, f"{csc}.logevent_errorCode", d1, d2))

In [ ]:
if csc_index > 0:
    fault_states = ss_df[(ss_df['summaryState'] == 3) & (ss_df[f'{csc}ID'] == csc_index)]
else:
    fault_states = ss_df[(ss_df['summaryState'] == 3)]

In [ ]:
timedelta = pd.Timedelta(time_window, unit='sec')

In [ ]:
if fault_states.index.size:
    for timestamp in fault_states.index:
        t1 = timestamp - timedelta
        t2 = timestamp + timedelta
        error_indexes = ec_df.index.to_numpy()
        index = np.logical_and(error_indexes > t1, error_indexes < t2)
        error = ec_df[index]
        print("######################################")
        print(f"Fault State: {error.errorCode.index.strftime('%Y-%m-%dT%H:%M:%S.%f')[0]}")
        print(f"Error Code: {error.errorCode.values[0]}")
        print(f"Error Report: {error.errorReport.values[0]}")
        print(f"Traceback: {error.traceback.values[0]}")
else:
    print(f"No fault states found for {csc}")